In [ ]:
import math
import time
import random
import copy

## Problema Dormitórios

In [ ]:
dormitorios = ['sao paulo', 'flamengo', 'coritiba', 'cruzeiro', 'fortaleza']

preferencias = {
    'amanda' : ('cruzeiro', 'coritiba'),
    'pedro' : ('sao paulo', 'fortaleza'),
    'marcos' : ('flamengo', 'sao paulo'),
    'priscila' : ('sao paulo', 'fortaleza'),
    'jessica' : ('flamengo', 'cruzeiro'),
    'paulo' : ('coritiba', 'fortaleza'),
    'fred' : ('fortaleza', 'flamengo'),
    'suzana' : ('cruzeiro', 'coritiba'),
    'laura' : ('cruzeiro', 'coritiba'),
    'ricardo' : ('coritiba', 'flamengo')
}

In [ ]:
dominio = [(0, (len(dormitorios) * 2) - i - 1) for i in range(len(dormitorios) * 2)]
dominio

In [ ]:
def print_vagas(solucao:list):
    vagas = []

    for i in range(len(dormitorios)):
        # deminstrar que para cada quarto haverá duas vagas
        # ex: 0, 0, 1, 1. Indo pelo indice dos dormitorios
        vagas += [i, i]

    for i in range(len(solucao)):
        # pega o numero do indice do dormitorio
        dormitorio = dormitorios[vagas[solucao[i]]]
        print(list(preferencias.keys())[i], dormitorio)
        # apos preencher a vaga, retira ela
        # limpa a lista de vagas
        del vagas[solucao[i]]

print_vagas(solucao=[6,1,2,1,2,0,2,2,0,0])

In [ ]:
def custo_dormitorio(solucao:list):
    vagas = []
    custo = 0

    for i in range(len(dormitorios)):
        # deminstrar que para cada quarto haverá duas vagas
        # ex: 0, 0, 1, 1. Indo pelo indice dos dormitorios
        vagas += [i, i]

    for i in range(len(solucao)):
        # pega o dormitorio na posicao i
        dormitorio = dormitorios[vagas[solucao[i]]]
        # recupera as preferencias da pessoa atual na posicao i
        pref = preferencias.get(list(preferencias.keys())[i])
        
        # primeiro quarto desejado
        if dormitorio == pref[0]:
            pass
        # segunda preferencia + 1 pen
        elif dormitorio == pref[1]:
            custo += 1
        # nenhuma das duas + 3 pen
        else:
            custo += 3

        del vagas[solucao[i]]

    return custo


custo_dormitorio(solucao=[6,1,2,1,2,0,2,2,0,0])